In [6]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, AveragePooling2D


In [2]:
filename='mpii_new.csv'

In [3]:
df = pd.read_csv(filename)

In [4]:
df

,ID,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,5,060111501.jpg,980,322,896,318,865,248,943,226,...,147,923,123,995,163,961,223,3.806403,curling,sports
1,7,002058449.jpg,918,456,659,518,713,413,979,288,...,270,877,247,1112,384,1012,489,8.280870,curling,sports
2,14,029122914.jpg,-1,-1,301,473,302,346,362,345,...,220,371,213,396,309,393,290,2.204083,"truck driving, loading and unloading truck, ty...",occupation
3,15,061185289.jpg,-1,-1,-1,-1,-1,-1,-1,-1,...,288,552,355,-1,-1,-1,-1,4.143112,"truck driving, loading and unloading truck, ty...",occupation
4,16,013949386.jpg,-1,-1,-1,-1,110,385,208,355,...,303,212,153,319,123,376,39,4.431105,"truck driving, loading and unloading truck, ty...",occupation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,17367,050240222.jpg,607,689,599,542,605,416,691,410,...,232,663,228,623,289,555,333,3.480000,pushing car,transportation
11427,17368,033474347.jpg,876,461,815,380,779,296,742,303,...,214,663,223,674,286,631,260,2.003677,pushing car,transportation
11428,17369,082650067.jpg,1038,656,950,504,903,328,831,337,...,204,729,220,704,332,675,269,3.235820,pushing car,transportation
11429,17370,072772110.jpg,562,617,558,539,553,452,511,458,...,377,490,378,470,411,461,414,1.494952,pushing car,transportation


In [7]:
train_slike = []
train_labele = []
folder_dir = "images_training"
for image in os.listdir(folder_dir):
    if image.endswith(".jpg"):
        img = Image.open(os.path.join(folder_dir, image))
        np.append(train_slike, np.array(img))

        rezultat = df[df['NAME'] == image]
        rezultat_numpy = rezultat.to_numpy()

X_train =  np.array(train_slike)
Y_train = np.array(train_labele)

In [8]:
test_slike = []
test_labele = []
folder_dir = "images_test"
for image in os.listdir(folder_dir):
    if image.endswith(".jpg"):
        img = Image.open(os.path.join(folder_dir, image))
        np.append(test_slike, np.array(img))

        rezultat = df[df['NAME'] == image]
        rezultat_numpy = rezultat.to_numpy()

X_test =  np.array(test_slike)
Y_test = np.array(test_labele)

In [9]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
class SliceLayer(tf.keras.layers.Layer):
    def __init__(self, slice_point, axis, **kwargs):
        super(SliceLayer, self).__init__(**kwargs)
        self.slice_point = slice_point
        self.axis = axis

    def call(self, inputs):
        slices = tf.split(inputs, num_or_size_splits=[self.slice_point, tf.shape(inputs)[self.axis] - self.slice_point], axis=self.axis)
        return slices


if __name__ == "__main__":
    input_tensor = tf.random.normal([1, 4, 368, 368]) 

    slice_layer = SliceLayer(slice_point=3, axis=1)

    output_tensors = slice_layer(input_tensor)

    for i, output_tensor in enumerate(output_tensors):
        print(f"Output tensor {i} shape: {output_tensor.shape}")
AveragePooling2D((9,9), 8)

In [ ]:
gaussian_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
constant_initializer = tf.keras.initializers.Constant(value=0.0)
model_s1 = Sequential([
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(32, (5,5), padding=2, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    Conv2D(512, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    Conv2D(512, (1,1), padding=0, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    Conv2D(10, (1,1), padding=0, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    ])

In [ ]:
gaussian_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
constant_initializer = tf.keras.initializers.Constant(value=0.0)
model_s2 = Sequential([
    AveragePooling2D((9,9), 8),
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(128, (9,9), padding=4, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    MaxPooling2D((3,3), 2),
    Conv2D(32, (5,5), padding=2, activation='relu', input_shape = (1,4,368,368), kernel_initializer=gaussian_initializer, bias_initializer=constant_initializer),
    ])

In [ ]:
model_concatenated = tf.keras.layers.Concatenate(axis=1)([model_s1, model_s2])
model = tf.keras.Model(inputs=[model_s1, model_s2], outputs=model_concatenated)